# Ptychography Tutorial 04

This is the fourth tutorial notebook in the iterative ptychography series.  
In this tutorial notebook we will cover:
- GPU memory-management for ptychographic reconstructions

### Downloads
This tutorial uses the following datasets:
- [ptycho_ducky_simulation_01.h5](https://drive.google.com/file/d/1VtYVHWiuI8AT0yucaylIcQQJ_px79ash/view?usp=drive_link)
- [ptycho_ducky_vacuum-probe_01.h5](https://drive.google.com/file/d/1CeVGVvMf2QeJK1ADQ9MBK308NhvoVaVn/view?usp=drive_link)

### Acknowledgements

This tutorial was created by the py4DSTEM `phase_contrast` team:
- Georgios Varnavides (gvarnavides@berkeley.edu)
- Stephanie Ribet (sribet@lbl.gov)
- Colin Ophus (clophus@lbl.gov)

Last updated: 2024 May 6

## Introduction

Ptychographic reconstructions can be very computationally expensive. py4DSTEM offers single-node GPU acceleration to speed up these calculations.

GPU calculations however, are often VRAM (memory) limited. Here, we investigate various flags as well as tips and tricks to enable efficient GPU memory management.

In [1]:
import numpy as np
import cupy as cp
import py4DSTEM

from py4DSTEM.process.phase.utils import get_array_module
from cupy.fft.config import get_plan_cache

print(py4DSTEM.__version__)
print(cp.__version__)

0.14.14
13.0.0


In [2]:
file_path = 'data/'
file_data = file_path + 'ptycho_ducky_simulation_01.h5'
file_probe = file_path + 'ptycho_ducky_vacuum-probe_01.h5'

probe = py4DSTEM.read(file_probe)
dataset = py4DSTEM.read(file_data)
dataset

DataCube( A 4-dimensional array of shape (33, 33, 200, 200) called 'datacube',
          with dimensions:

              Rx = [0.0,5.0,10.0,...] A
              Ry = [0.0,5.0,10.0,...] A
              Qx = [0.0,0.025,0.05,...] A^-1
              Qy = [0.0,0.025,0.05,...] A^-1
)

### GPU Memory Usage

First, let's define two helper functions to:  
- clear a class's cupy arrays
- report class GPU memory usage

In [3]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
cache = get_plan_cache()

def report_class_gpu_memory_usage(cls):

    size_on_device = 0
    for att in sorted(cls.__dict__):
        attr = getattr(cls,att)
        
        if isinstance(attr,(np.ndarray,cp.ndarray)):
            
            if get_array_module(attr) is not np:
                size_on_device += attr.nbytes
                print(f"Attribute {att}: {attr.nbytes*1e-3:.3f}Kb")
    
        if isinstance(attr,list) and isinstance(attr[0],(np.ndarray,cp.ndarray)):
            
            size_entering_loop = size_on_device
            for el in attr:
                if get_array_module(el) is not np:
                    size_on_device += el.nbytes
                    
            print(f"Attribute {att}: {(size_on_device-size_entering_loop)*1e-3:.3f}Kb")
    
    print(f"\nEstimated size on device: {size_on_device*1e-6:.3f}Mb")
    print(f"Reported size on device: {mempool.used_bytes()*1e-6:.3f}Mb")
    
    return None

def release_class_gpu_memory(clear_fft_cache = True):

    if clear_fft_cache is True:
        cache.clear()
        
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()

    return None

### CPU-only calculation

To estabish a baseline, we perform a CPU-only calculation:

In [4]:
ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='cpu', # default
).preprocess(
    plot_rotation=False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
    store_initial_arrays= False, # don't store arrays necessary for reset=True
)

report_class_gpu_memory_usage(ptycho)


Estimated size on device: 0.000Mb
Reported size on device: 0.000Mb


As expected, preprocessing with `device='cpu'` uses no GPU memory.  
Also notice two additional flags to help with memory issues:
- `vectorized_com_calculation=False` &rarr; computes the otherwise memory-expensive CoM calculation using a for-loop
- `store_initial_arrays=False` &rarr; doesn't store the arrays necessary to restart the calculation

`reconstruct` will similarly work directly on the CPU and use no GPU memory:

In [5]:
ptycho = ptycho.reconstruct(
    seed_random=0, 
    num_iter = 2,
    max_batch_size=512,
)

report_class_gpu_memory_usage(ptycho)

Reconstructing object and probe: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.12s/ iter]


Estimated size on device: 0.000Mb
Reported size on device: 0.000Mb


### Expensive calculations on GPU

The next step is to store the 4D-data and other necessary utility arrays on the CPU, but perform the actual computations on the GPU by "streaming" the necessary data inside the loop. This can be enabled by specifying `device='gpu'` and `storage='cpu'`.

In [7]:
ptycho = None
release_class_gpu_memory()

ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='gpu', # perform calculations on the GPU
    storage='cpu', # store data and other utility arrays on CPU
).preprocess(
    plot_rotation=False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
    store_initial_arrays= False, # don't store arrays necessary for reset=True
)

report_class_gpu_memory_usage(ptycho)

Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _probe: 320.000Kb

Estimated size on device: 11.779Mb
Reported size on device: 11.780Mb


As we can see this only stores the minimum necessary arrays on the GPU (notably the object and probe arrays), and releases all other intermediate memory after execution (seen by the estimated size above matches the reported size by cupy).  

In [8]:
ptycho = ptycho.reconstruct(
    seed_random=0, 
    num_iter = 2,
    max_batch_size=512,
)

report_class_gpu_memory_usage(ptycho)

Reconstructing object and probe: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.54 iter/s]


Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _probe: 320.000Kb

Estimated size on device: 11.779Mb
Reported size on device: 11.780Mb


### FFT Caching

In addition to intermediate arrays which are deallocated and cleared above, the other big culprit in the GPU storing more arrays than presumably requested is cupy's default FFT plan caching.

In [9]:
ptycho = None
release_class_gpu_memory(clear_fft_cache=False)

ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='gpu', # perform calculations on the GPU
    storage='cpu', # store data and other utility arrays on CPU
    clear_fft_cache = False, # don't clear FFT cache at the end
).preprocess(
    plot_rotation = False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
    store_initial_arrays = False, # don't store arrays necessary for reset=True
)

report_class_gpu_memory_usage(ptycho)

Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _probe: 320.000Kb

Estimated size on device: 11.779Mb
Reported size on device: 11.789Mb


Notice that when we don't specify `clear_fft_cache=True` (on by default), the GPU reports some additional memory usage. These are the cached FFT plans, which you can inspect as follows:

In [10]:
cache.show_info()

------------------- cuFFT plan cache (device 0) -------------------
cache enabled? True
current / max size   : 2 / 16 (counts)
current / max memsize: 9728 / (unlimited) (bytes)
hits / misses: 2 / 2 (counts)

cached plans (most recently used first):
key: ((200, 200), (200, 200), 1, 40000, (200, 200), 1, 40000, 41, 1089, 'C', 2, None), plan type: PlanNd, memory usage: 9216
key: ((200, 200), (200, 200), 1, 1, (200, 200), 1, 1, 41, 1, 'C', 1, None), plan type: PlanNd, memory usage: 512



By default, we enable cupy's default FFT plan caching within each function call (since it is quite performant), but clear the cache at the end of `preprocess`, `reconstruct`, and `visualize`. An even more aggresive solution to allow better memory management still is to limit the cache size, or disable it completely (by specifying a size of zero).

In [11]:
cache.set_size(0)

ptycho = None
release_class_gpu_memory()

ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='gpu', # perform calculations on the GPU
    storage='cpu', # store data and other utility arrays on CPU
    clear_fft_cache=False, # for demonstration purposes that cache is not used
).preprocess(
    plot_rotation=False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
    store_initial_arrays= False, # don't store arrays necessary for reset=True
).reconstruct(
    seed_random=0, 
    num_iter = 2,
    max_batch_size=512,
)

report_class_gpu_memory_usage(ptycho)

Reconstructing object and probe: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.78 iter/s]

Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _probe: 320.000Kb

Estimated size on device: 11.779Mb
Reported size on device: 11.780Mb


In [12]:
cache.show_info()

------------------- cuFFT plan cache (device 0) -------------------
cache enabled? False
current / max size   : 0 / 0 (counts)
current / max memsize: 0 / (unlimited) (bytes)
hits / misses: 0 / 0 (counts)

cached plans (most recently used first):



### Store/compute all arrays on GPU  
If memory is not a concern, then you can skip the `storage='cpu'` flag and store all arrays on the GPU for the best performance.

In [13]:
cache.set_size(16) # restore to default caching

ptycho = None
release_class_gpu_memory()

ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='gpu', # perform calculations on the GPU
    storage='gpu', # can be omited, defaults to 'device'
).preprocess(
    plot_rotation=False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
    store_initial_arrays= False, # don't store arrays necessary for reset=True
).reconstruct(
    seed_random=0, 
    num_iter = 2,
    max_batch_size=512,
)

report_class_gpu_memory_usage(ptycho)

Reconstructing object and probe: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 10.46 iter/s]


Attribute _amplitudes: 174240.000Kb
Attribute _com_fitted_x: 4.356Kb
Attribute _com_fitted_y: 4.356Kb
Attribute _com_measured_x: 4.356Kb
Attribute _com_measured_y: 4.356Kb
Attribute _com_normalized_x: 4.356Kb
Attribute _com_normalized_y: 4.356Kb
Attribute _com_x: 4.356Kb
Attribute _com_y: 4.356Kb
Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _positions_initial: 8.712Kb
Attribute _positions_px: 8.712Kb
Attribute _positions_px_initial: 8.712Kb
Attribute _positions_px_initial_com: 0.008Kb
Attribute _probe: 320.000Kb

Estimated size on device: 186.080Mb
Reported size on device: 186.085Mb


Notice this reports a lot more arrays on the GPU, notably the amplitude data itself - which can be quite large.

### Function-specific `device`

Note that you can overwrite the top-level `device` and `clear_fft_cache` attributes set at initialization during each subsequent function call, meaning you can e.g. preprocess your data on the 'cpu' and reconstruct on the 'gpu':

In [14]:
ptycho = None
release_class_gpu_memory()

ptycho = py4DSTEM.process.phase.SingleslicePtychography(
    datacube=dataset, 
    energy=80e3, 
    defocus=500, 
    vacuum_probe_intensity=probe.data,
    verbose = False,
    device='cpu', # preprocess on the CPU
).preprocess(
    plot_rotation=False,
    plot_center_of_mass = False,
    plot_probe_overlaps = False,
    vectorized_com_calculation = False, # disable default CoM vectorized calc
).reconstruct(
    seed_random=0, 
    num_iter = 2,
    max_batch_size=512,
    device='gpu', # reconstruct on the GPU
)

report_class_gpu_memory_usage(ptycho)

Reconstructing object and probe: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.66 iter/s]

Attribute _known_aberrations_array: 320.000Kb
Attribute _object: 11139.200Kb
Attribute _object_initial: 11139.200Kb
Attribute _probe: 640.000Kb
Attribute _probe_initial: 640.000Kb
Attribute _probe_initial_aperture: 320.000Kb

Estimated size on device: 24.198Mb
Reported size on device: 24.199Mb


### `max_batch_size`

Finally, you can use the `max_batch_size` in both `preprocess` and `reconstruct` to ensure the number of probe-positions streamed to `device` at any given iteration is kept small-enough for your GPU memory to handle.

Note: this option currently defaults to "mini-batch" behaviour in stochastic gradient descent, meaning the object and probes will be updated per batch, not per iteration.